In [3]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [5]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [6]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-2.8209408841343304 155.73404488630678
-11.438922096324504 175.06817546281383
-48.74201247834004 -115.9453931648619
26.417566370312613 -80.9305520553104
-49.27903422142353 -23.626253088328156
-9.122734304895815 103.73346858077753
-64.14100334991552 -51.26410535785388
28.147123924409556 -12.891447077135723
1.1297442697660927 92.82523939141294
-58.23211845478835 32.55582713178774
-68.56794873046151 -89.00898111016609
36.898957826351435 -109.31379293648214
81.89044090001971 -61.79988691723884
-72.28847253027371 163.87693838266523
-54.26940286887963 34.369488116138456
-73.1604739521109 -95.44906535348906
-44.25456407873363 155.29948307570004
49.56976085196092 -134.92519589358827
30.91603326611319 -97.57175668272252
74.99129768727218 -84.1738931532942
25.7962671825957 17.869343774939153
-70.32806084301589 9.949813286676147
-35.180732191427445 32.80726395069743
-31.59567059095224 -175.50341376288924
-51.79232133968124 68.79818478225454
70.0138789785342 75.4605323571179
-9.224093746369206 -17

In [7]:
from citipy import citipy

In [8]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

605